# Preamble

Singapore is a country in Southeast Asia with a parliamentary democracy. It has a single chamber parliament comprising _.

In [ ]:
import pandas as pd
import requests
import re
import datetime
import bs4

In [ ]:
PAGE = "https://www.parliament.gov.sg/parliamentary-business/votes-and-proceedings?parliament=14&fromDate=&toDate=&page=1&pageSize=100"

dates_source = requests.get(PAGE).text
date_pattern = r"\d{1,2}\s\w+\s\d{4}"
dates = re.findall(date_pattern, dates_source)
dates

In [ ]:
len(dates)

In [ ]:
dates_df = (pd.DataFrame(dates, columns=['sitting_dates'])
            .assign(sitting_dates = lambda x: pd.to_datetime(x.sitting_dates),
                    url = lambda x: ['https://sprs.parl.gov.sg/search/#/fullreport?sittingdate=' + str(datetime.datetime.strftime(x[0], "%d-%m-%Y")) for x in zip(x.sitting_dates)]
                    )
            )

In [ ]:
dates_df

In [ ]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    # Already added when you pass json=
    # 'Content-Type': 'application/json',
    'Origin': 'https://sprs.parl.gov.sg',
    'Referer': 'https://sprs.parl.gov.sg/search/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

params = {
    'sittingDate': '04-10-2022',
}

json_data = {
    'headers': {
        'normalizedNames': {},
        'lazyUpdate': None,
    },
}

response = requests.post('https://sprs.parl.gov.sg/search/getHansardReport/', params=params, headers=headers, json=json_data)

In [ ]:
response.json().get('attendanceList')